In [1]:
%load_ext dotenv
%dotenv

In [2]:
# !pip install --upgrade langchain langchain-community langchain-google-genai 
# !pip install -U langchain-groq

In [3]:
from ipywidgets import widgets, interact, interactive, Layout
from IPython.display import display, Markdown
import os
from langchain_google_genai import GoogleGenerativeAI
from langchain_groq import ChatGroq  
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [4]:
class LLMAPI:
    def __init__(self, provider="google", model="gemini-1.5-flash", api_key=None):
        self.provider = provider
        self.model = model
        self.api_key = api_key or os.getenv("GOOGLE_API_KEY")
        self.llm = self.initialize_llm()

    def initialize_llm(self):
        if self.provider == "google":
            return GoogleGenerativeAI(model=self.model, google_api_key=self.api_key)
        elif self.provider == "groq":
            return ChatGroq(model=self.model, api_key=self.api_key)
        else:
            raise ValueError("Unsupported LLM provider")

    def get_response(self, user_message):
        response = conversation_chain.invoke(input=user_message)
        return response['response'].strip()

# Initialize the LLM API
llm_api = LLMAPI(provider="groq", model="llama-3.1-70b-versatile", api_key=os.getenv("GROQ_API_KEY"))

# Initialize memory to retain conversation history
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

# Create a conversation chain with the model and memory
conversation_chain = ConversationChain(llm=llm_api.llm, memory=memory)

# Create the chat area
chat_area = widgets.Output(layout={'border': '1px solid black', 'height': '300px', 'overflow': 'auto'})

# Create the input field with a larger layout
input_field = widgets.Textarea(placeholder='Type your message here...',
                              layout=Layout(width='100%', height='80px'))

# Create the send button
send_button = widgets.Button(description='Send')

# Arrange the widgets in a layout
layout = widgets.VBox([chat_area, widgets.HBox([input_field, send_button])])

def send_message(sender):
    user_message = input_field.value
    input_field.value = ''
    
    with chat_area:
        # user_html = markdown2.markdown(f'**You:** {user_message}')
        user_html = f'**You:** {user_message}'
        display(Markdown(user_html))
    
    # Get the model's response
    agent_response = llm_api.get_response(user_message)
    
    with chat_area:
        # agent_html = markdown2.markdown(f'**Agent:** {agent_response}')
        agent_html = f'**Agent:** {agent_response}'
        display(Markdown(agent_html))

send_button.on_click(send_message)

display(layout)

/home/rajesh/Desktop/repos/experimental/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(
